Support vector machines are capable of performing non-linear classification, regression, and outliner detection. They are well-suited for complex small to medium sized data sets.

For this example we will be using Riot's API to create the data-set. Due to the request limitations on the API-key, we will be streaming the data.

In [4]:
import requests
import apiKey
import numpy as np
import matplotlib.pyplot as plt


region1: str = "jp1"
region2: str = "asia"
summonerId: str = apiKey.Id
apiKey = apiKey.Key

class RankedMatches:
    def __init__(self, region1: str, region2: str, summonerId: str, apiKey: str):
        self.puuid = self.retrievePuuid(region1, summonerId, apiKey)
        self.ListOfRankedMatches = self.retrieveListOfRankedMatches(region2, self.puuid, apiKey)
        self.data = self.matchData(region2, self.ListOfRankedMatches, apiKey)


    # Need puuid to retrieve match list for a summoner
    def retrievePuuid(self, region: str, summonerId: str, apiKey: str) -> str:
        ''''Retrieves puuid from Riot's API'''
        summonerDataURL: str = "https://" + region + ".api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerId + "?api_key=" + apiKey
        return requests.get(summonerDataURL).json()["puuid"]

    # Acceptable regions are asia, america, etc
    def retrieveListOfRankedMatches(self, region: str, puuid: str, apiKey: str) -> list:
        '''Retrieves list of ranked matches for a player'''
        summonerMatchesURL: str = "https://" + region + ".api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuid + "/ids?api_key=" + apiKey
        return requests.get(summonerMatchesURL, params= {"type": "ranked", "count": "40"}).json()


    # Acceptable regions are asia, america, etc
    def matchData(self, region: str, matches: list, apiKey: str) -> list:
        '''Returns an array of match data, with n features and binary label'''
        matchList = []
        for match in matches:
            matchDataURL: str = "https://" + region + ".api.riotgames.com/lol/match/v5/matches/" + match + "?api_key=" + apiKey
            matchData = requests.get(matchDataURL).json()
            matchInfo = matchData["info"]
            participants = matchInfo["participants"]
            for participant in participants:
                if participant["summonerName"] == summonerId:
                    # Besides win, other valid parameters are <kills, deaths, championName, damageDealtToObjectives,
                    # dragonKills, timePlayed, totalDamageDealtToChampions, totalTimeCCDealt, visionScore, totalMinionsKilled>
                    if participant["win"]:
                        matchList.append([match, participant["totalMinionsKilled"], participant["totalDamageDealtToChampions"], participant["timePlayed"], participant["deaths"], participant["kills"], participant["visionScore"], 1])
                    else:
                        matchList.append([match, participant["totalMinionsKilled"], participant["totalDamageDealtToChampions"], participant["timePlayed"], participant["deaths"], participant["kills"], participant["visionScore"], 0])
        return matchList


myMatchList = RankedMatches(region1, region2, summonerId, apiKey)
print(myMatchList.data)


[['JP1_355539882', 171, 17803, 2076, 5, 2, 37, 1], ['JP1_355558489', 127, 7305, 1213, 2, 3, 13, 0], ['JP1_355547802', 137, 16759, 1440, 6, 5, 14, 0], ['JP1_355546905', 158, 11603, 1508, 4, 5, 15, 0], ['JP1_355555629', 202, 27559, 2000, 6, 9, 13, 0], ['JP1_355555053', 55, 4084, 624, 0, 4, 3, 0], ['JP1_355534745', 137, 13023, 1477, 7, 4, 9, 0], ['JP1_355533820', 204, 18479, 1924, 5, 2, 20, 0], ['JP1_355540659', 166, 29060, 1981, 3, 10, 26, 1], ['JP1_355540147', 216, 11988, 1777, 2, 2, 15, 1], ['JP1_355519498', 32, 7269, 1996, 4, 5, 67, 1], ['JP1_355509246', 155, 6160, 1303, 2, 2, 13, 0], ['JP1_355528070', 127, 8860, 1434, 1, 5, 13, 1], ['JP1_355527543', 223, 16209, 1844, 1, 2, 28, 1], ['JP1_355473005', 187, 29747, 2371, 9, 7, 27, 0], ['JP1_355472062', 149, 15988, 1547, 1, 6, 16, 1], ['JP1_355480831', 210, 20830, 2462, 10, 4, 21, 1], ['JP1_355480140', 166, 9862, 1626, 5, 0, 15, 0], ['JP1_355459335', 223, 35239, 2222, 9, 10, 19, 0], ['JP1_355448651', 171, 11611, 1729, 3, 2, 18, 1], ['JP1_3